# 月报生成器
# 妈的，我TM简直是个天才（手动狗头）

### 单位换算
亿千瓦=10,000万千瓦，万千瓦=10兆瓦   
万千瓦 = $10^4 \times 10^3 Wh$  
亿千瓦 = $10^4 \times 10^4 \times 10^3 Wh$  
兆瓦 = $10^6 Wh$

In [121]:
import numpy as np
import pandas as pd
import copy as cp
from NewEnergyDeparmentFileParse import *

In [15]:
def NameSetCheck(set1, set2):
    if set1 != set2:
        diff1 = set1 - set2
        diff2 = set2 - set1
        print('名称集合不相等！请检查是否有风电场名称变更')
        print('前者中为：' + str(diff1) )
        print('后者中为：' + str(diff2) )
        raise('Name Set Mismatch')

In [44]:
Year = 2018
Month = 7
# Macro define: file names
## Input Files
FileNewEnergyDeparment = '【1】' + str(Year) + '年' + str(Month) + '月新能源利用小时.xlsx'  # 新能源处提供的《利用小时数》，利用小时数以这个为准
FileUtilHrsOMS = '【2】lyxssExcel.xlsx'  # OMS系统中的《利用小时数》，仅用这个文件中的弃风电量，其利用小时数不准的。
FileLastMonthReport = '【3】上月报表.xlsx'  #获取上月累计发电量、累计弃风电量，用于与本月数据累加
FileNameDict = '【4】NameDict.xlsx'  # 月报中风电场名称、新能源处利用小时数中风电场名称，等各种文件中不同风电场名称的对应表格。
FileNameDictGenerationGroup = '【5】NameDictGenerationGroup.xlsx'  # 发电集团汇总表格中发电集团顺序
FileNewEnergyDeparmentLastYear = '【6】' + str(Year-1) + '年' + str(Month) + '月新能源利用小时.xlsx'  # 去年同期新能源处提供的《利用小时数》，用于同比比较
FileNewEnergyDeparmentLastMonth = '【7】' + str(Year) + '年' + str(Month-1) + '月新能源利用小时.xlsx'  # 上个月新能源处提供的《利用小时数》，用于环比比较

## Output Files
FileAppendix1 = r'【Output】附表1-冀北电网并网风电场发电情况'
FileTable10 = r'【Output】表10-风电发电集团发电量，利用小时数'

ColumnsNewEnergyDeparment = ['装机容量\n（兆瓦）', str(Month) + '月发电量\n(万千瓦时)', '年累计利用小时']

# 1、场站名称对照表导入

In [17]:
df_NameDict = pd.read_excel(FileNameDict, index_col=0)
# df_NameDict

# 2、OMS文件读入（获取本月弃风电量）

In [18]:
# 文件导入
df_UtilHrsOMS = pd.read_excel(FileUtilHrsOMS, skiprows=1)
# 名称一致性检查
NameSetCheck(set(df_NameDict['风电场名称（OMS）'].tolist()), set(df_UtilHrsOMS['风电场名称']) )
# 导入后处理：修改列名，设定index列
df_UtilHrsOMS = df_UtilHrsOMS.set_index('风电场名称')
df_UtilHrsOMS = df_UtilHrsOMS[['弃风电量']]/10.0  # OMS系统中的弃风电量单位是MWh，改成（万千瓦时）需要除以10
df_UtilHrsOMS.columns = ['本月弃风电量']
df_UtilHrsOMS.index.name = '风电场名称（OMS）'
# df_UtilHrsOMS

# 3、上次月报结果导入

In [19]:
# 文件导入
df_LastMonthReport = pd.read_excel(FileLastMonthReport)
# 名称一致性检查
NameSetCheck(set(df_NameDict['风电场名称（月报）'].tolist()), set(df_LastMonthReport['风电场']) )
# 导入后处理
df_LastMonthReport['发电量'] = pd.to_numeric(df_LastMonthReport['发电量'], errors='coerce')
df_LastMonthReport['利用小时数'] = pd.to_numeric(df_LastMonthReport['利用小时数'], errors='coerce')
df_LastMonthReport['弃风电量'] = pd.to_numeric(df_LastMonthReport['弃风电量'], errors='coerce')
df_LastMonthReport['弃风率(%)'] = pd.to_numeric(df_LastMonthReport['弃风率(%)'], errors='coerce')
df_LastMonthReport = df_LastMonthReport.set_index('风电场')
df_LastMonthReport = df_LastMonthReport[ ['发电量', '利用小时数', '弃风电量', '弃风率(%)'] ]
df_LastMonthReport.columns = ['上月年累计发电量', '上月年累计利用小时数', '上月年累计弃风电量', '上月年累计弃风率'] #修改列名，否则会与OMS里的列名重复冲突
df_LastMonthReport.index.name = '风电场名称（月报）'
# df_LastMonthReport

# 4、新能源处利用小时数表格导入

In [20]:
# 抽取新能源处利用小时数表格：获取本月发电量、本月累计利用小时数
lst_Sites = df_NameDict['风电场名称（新能源处）'].tolist()
if len(lst_Sites) != len(GetItemLoc(lst_Sites ,FileNewEnergyDeparment)):
    print('场站对照表与新能源处文件中的场站数量不一致，请检查是否有场站数量或名称变化！')
    raise('Name Dict Mismatch')

lst_Columns = ColumnsNewEnergyDeparment
df_NewEnergyDeparment = GetTable(lst_Sites, lst_Columns, FileNewEnergyDeparment)
df_NewEnergyDeparment.columns = ['装机容量', '本月发电量', '本月年累计利用小时']
df_NewEnergyDeparment.index.name = '风电场名称（新能源处）'
df_NewEnergyDeparment['装机容量'] = df_NewEnergyDeparment['装机容量']/10.0 # 新能源处的装机容量是MW，改成（万千瓦）需要除以10

# 5、附表1生成

In [21]:
tmp1 = df_NameDict.join(df_LastMonthReport, on='风电场名称（月报）') 
tmp2 = tmp1.join(df_UtilHrsOMS, on='风电场名称（OMS）')
df_Combined = tmp2.join(df_NewEnergyDeparment, on='风电场名称（新能源处）')
df_Combined

,风电场名称（月报）,风电场名称（新能源处）,风电场名称（OMS）,发电集团,地区,上月年累计发电量,上月年累计利用小时数,上月年累计弃风电量,上月年累计弃风率,本月弃风电量,装机容量,本月发电量,本月年累计利用小时
月报序号,,,,,,,,,,,,,
1,莲花滩,莲花滩风电场,莲花滩风电场,河北建投,张家口,56060.6,1403.240000,10948.1,0.163383,0.00,39.950,6381.70,1562.980000
2,建投明桂沟,建投明桂沟风电场,建投明桂沟,河北建投,承德,44340.5,1777.500040,1048.5,0.023100,0.00,24.945,2514.60,1878.300040
3,如意河,如意河风电场,如意河风电场,河北建投,承德,32835.5,1645.864637,848.3,0.025184,0.00,19.950,1754.00,1733.774637
4,建投魁通沟,建投魁通沟风电场,建投魁通沟,河北建投,承德,43956.7,1766.016935,426.9,0.009618,6.80,24.890,3020.30,1887.356935
5,长风,长风风电场,长风风电场,河北建投,承德,17222.0,1148.120000,642.5,0.035965,131.00,15.000,967.00,1212.580000
6,秋林,秋林风电场,秋林风电场,河北建投,张家口,18957.9,1265.530000,2038.1,0.097071,25.50,14.980,1388.00,1358.180000
7,月牙,建投月牙风电场,建投月牙,河北建投,承德,17169.4,1734.264444,300.5,0.017201,0.00,9.900,1284.80,1864.034444
8,清三营,清三营风电场,清三营风电场,河北建投,张家口,11550.5,1171.429817,46570.0,0.801266,84.00,9.860,742.60,1246.739817
9,五花坪,五花坪风电场,五花坪风电场,河北建投,张家口,9401.7,1173.719488,2089.4,0.181828,41.50,8.010,992.30,1297.599488


In [22]:
df_Combined['本月年累计发电量'] = df_Combined['上月年累计发电量'] + df_Combined['本月发电量']
df_Combined['本月年累计弃风电量'] = df_Combined['上月年累计弃风电量'] + df_Combined['本月弃风电量']
df_Combined['本月年累计弃风率'] = df_Combined['本月年累计弃风电量']/(df_Combined['本月年累计弃风电量'] + df_Combined['本月年累计发电量'])
df_Combined

,风电场名称（月报）,风电场名称（新能源处）,风电场名称（OMS）,发电集团,地区,上月年累计发电量,上月年累计利用小时数,上月年累计弃风电量,上月年累计弃风率,本月弃风电量,装机容量,本月发电量,本月年累计利用小时,本月年累计发电量,本月年累计弃风电量,本月年累计弃风率
月报序号,,,,,,,,,,,,,,,,
1,莲花滩,莲花滩风电场,莲花滩风电场,河北建投,张家口,56060.6,1403.240000,10948.1,0.163383,0.00,39.950,6381.70,1562.980000,62442.30,10948.10,0.149176
2,建投明桂沟,建投明桂沟风电场,建投明桂沟,河北建投,承德,44340.5,1777.500040,1048.5,0.023100,0.00,24.945,2514.60,1878.300040,46855.10,1048.50,0.021888
3,如意河,如意河风电场,如意河风电场,河北建投,承德,32835.5,1645.864637,848.3,0.025184,0.00,19.950,1754.00,1733.774637,34589.50,848.30,0.023938
4,建投魁通沟,建投魁通沟风电场,建投魁通沟,河北建投,承德,43956.7,1766.016935,426.9,0.009618,6.80,24.890,3020.30,1887.356935,46977.00,433.70,0.009148
5,长风,长风风电场,长风风电场,河北建投,承德,17222.0,1148.120000,642.5,0.035965,131.00,15.000,967.00,1212.580000,18189.00,773.50,0.040791
6,秋林,秋林风电场,秋林风电场,河北建投,张家口,18957.9,1265.530000,2038.1,0.097071,25.50,14.980,1388.00,1358.180000,20345.90,2063.60,0.092086
7,月牙,建投月牙风电场,建投月牙,河北建投,承德,17169.4,1734.264444,300.5,0.017201,0.00,9.900,1284.80,1864.034444,18454.20,300.50,0.016023
8,清三营,清三营风电场,清三营风电场,河北建投,张家口,11550.5,1171.429817,46570.0,0.801266,84.00,9.860,742.60,1246.739817,12293.10,46654.00,0.791455
9,五花坪,五花坪风电场,五花坪风电场,河北建投,张家口,9401.7,1173.719488,2089.4,0.181828,41.50,8.010,992.30,1297.599488,10394.00,2130.90,0.170133


In [23]:
df_TableAppendix1 = df_Combined[ ['风电场名称（月报）', '本月年累计发电量', '本月年累计利用小时', '本月年累计弃风电量', '本月年累计弃风率'] ]
df_TableAppendix1

,风电场名称（月报）,本月年累计发电量,本月年累计利用小时,本月年累计弃风电量,本月年累计弃风率
月报序号,,,,,
1,莲花滩,62442.30,1562.980000,10948.10,0.149176
2,建投明桂沟,46855.10,1878.300040,1048.50,0.021888
3,如意河,34589.50,1733.774637,848.30,0.023938
4,建投魁通沟,46977.00,1887.356935,433.70,0.009148
5,长风,18189.00,1212.580000,773.50,0.040791
6,秋林,20345.90,1358.180000,2063.60,0.092086
7,月牙,18454.20,1864.034444,300.50,0.016023
8,清三营,12293.10,1246.739817,46654.00,0.791455
9,五花坪,10394.00,1297.599488,2130.90,0.170133


In [24]:
df_TableAppendix1.to_excel(FileAppendix1 + '.xlsx')

# 6、发电集团表10生成

In [25]:
df_GenerationGroup = df_Combined[ ['风电场名称（月报）', '发电集团', '装机容量', '本月年累计发电量', '本月年累计弃风电量'] ]
df_GenerationGroup = df_GenerationGroup.pivot_table(index=['发电集团'], values=['装机容量', '本月年累计发电量', '本月年累计弃风电量'], aggfunc=np.sum)
df_TableGenerationGroup = pd.read_excel(FileNameDictGenerationGroup, index_col=0)
df_TableGenerationGroup = df_TableGenerationGroup.join(df_GenerationGroup, on='发电集团')

JiBeiCapcity = np.sum(df_TableGenerationGroup['装机容量']) ## 总装机容量，用于求各发电集团装机占比
df_TableGenerationGroup['装机占比'] = df_TableGenerationGroup['装机容量'] / JiBeiCapcity
df_TableGenerationGroup['平均利用小时'] = df_TableGenerationGroup['本月年累计发电量'] / df_TableGenerationGroup['装机容量']
JiBeiUtilHrs = GetTable(['全网风电'], ['年累计利用小时'], FileNewEnergyDeparment).values[0][0] ## 冀北本月年累计利用小时数，用于求各集团与冀北的偏差
df_TableGenerationGroup['冀北偏差'] = df_TableGenerationGroup['平均利用小时'] - JiBeiUtilHrs
df_TableGenerationGroup['弃风率'] = df_TableGenerationGroup['本月年累计弃风电量'] / (df_TableGenerationGroup['本月年累计弃风电量'] + df_TableGenerationGroup['本月年累计发电量'])
# 列重新排序
df_TableGenerationGroup = df_TableGenerationGroup[ ['发电集团', '装机容量', '装机占比', '本月年累计发电量', '平均利用小时', '冀北偏差', '本月年累计弃风电量', '弃风率'] ]
# 重命名列名，使之与月报表10相同
df_TableGenerationGroup.columns = ['发电集团', '装机容量', '装机占比', '发电量', '平均利用小时', '冀北偏差', '弃风电量', '弃风率']
df_TableGenerationGroup

,发电集团,装机容量,装机占比,发电量,平均利用小时,冀北偏差,弃风电量,弃风率
序号,,,,,,,,
1,河北建投,190.135,0.167384,305670.20,1607.648250,164.702215,68840.90,0.183815
2,龙源电力,116.950,0.102956,155836.50,1332.505344,-110.440691,11625.00,0.069419
3,国华,102.475,0.090213,133386.40,1301.648207,-141.297828,13663.80,0.092919
4,中节能,69.350,0.061052,98156.40,1415.377073,-27.568962,11005.50,0.100818
5,华源电力,63.900,0.056254,90625.90,1418.245696,-24.700339,13171.40,0.126895
6,华电国际,55.050,0.048463,89429.00,1624.504995,181.558961,5116.00,0.054112
7,国电电力,54.750,0.048199,106233.30,1940.334247,497.388212,3097.80,0.028334
8,红松,54.690,0.048146,73777.70,1349.016274,-93.929761,4647.30,0.059258
9,大唐国际,48.900,0.043049,67362.50,1377.556237,-65.389798,423.20,0.006243


In [26]:
df_TableGenerationGroup.to_excel(FileTable10 + '.xlsx')

# 月报各分表计算

In [27]:
df_Combined.columns

Index(['风电场名称（月报）', '风电场名称（新能源处）', '风电场名称（OMS）', '发电集团', '地区', '上月年累计发电量',
       '上月年累计利用小时数', '上月年累计弃风电量', '上月年累计弃风率', '本月弃风电量', '装机容量', '本月发电量',
       '本月年累计利用小时', '本月年累计发电量', '本月年累计弃风电量', '本月年累计弃风率'],
      dtype='object')

### 1）表2-本月发电量

In [74]:
df_Tab2_col1 = df_Combined.groupby(['地区']).sum()[['本月发电量']]/10000.0  # 获得本月发电量（亿千瓦时）

In [126]:
# 建立合并对照表头，“地区”字段用于显示，“地区平均”字段用于合并从《新能源处利用小时数》抽取的数据
df_TabIndex = pd.DataFrame({'地区': ['张家口', '承德', '唐山', '秦皇岛'], '地区平均': ['张家口平均', '承德平均', '唐山平均', '秦皇岛平均']}, \
                           index=[1, 2, 3, 4])


# 获取去年同期数据
df_Tab2_col2 = GetTable(['张家口平均', '承德平均', '唐山平均', '秦皇岛平均'], [str(Month)+'月发电量\n（万千瓦时）'], FileNewEnergyDeparmentLastYear)
df_Tab2_col2 = df_Tab2_col2 / 10000.0
df_Tab2_col2.columns = [str(Year-1)+'年'+str(Month)+'月份发电量']

# 获取上月环比数据
df_Tab2_col3 = GetTable(['张家口平均', '承德平均', '唐山平均', '秦皇岛平均'], [str(Month-1)+'月发电量\n(万千瓦时)'], FileNewEnergyDeparmentLastMonth)
df_Tab2_col3 = df_Tab2_col3 / 10000.0
df_Tab2_col3.columns = [str(Year)+'年'+str(Month-1)+'月份发电量']

# 表格组合
df_Tab2_tmp = df_TabIndex.join(df_Tab2_col1, on='地区').join(df_Tab2_col2, on='地区平均').join(df_Tab2_col3, on='地区平均')

# 计算同比、环比涨幅
df_Tab2_tmp[str(Year-1)+'年'+str(Month)+'月份同比（%）'] = 100*(df_Tab2_tmp['本月发电量'] - df_Tab2_tmp[str(Year-1)+'年'+str(Month)+'月份发电量']) /\
                                                            df_Tab2_tmp[str(Year-1)+'年'+str(Month)+'月份发电量']
df_Tab2_tmp[str(Year)+'年'+str(Month-1)+'月份环比（%）'] = 100*(df_Tab2_tmp['本月发电量'] - df_Tab2_tmp[str(Year)+'年'+str(Month-1)+'月份发电量']) /\
                                                            df_Tab2_tmp[str(Year)+'年'+str(Month-1)+'月份发电量']

# 新增求和行，“冀北合计”
df_Tab2_tmp.loc[5] = df_Tab2_tmp.apply(lambda x: x.sum())
df_Tab2_tmp.loc[5, '地区'] = '冀北合计'
df_Tab2_tmp.loc[5, '地区平均'] = '冀北合计'

# 计算“冀北合计”的同比与环比
df_Tab2_tmp.loc[5, str(Year-1)+'年'+str(Month)+'月份同比（%）'] = 100*(df_Tab2_tmp.loc[5, '本月发电量'] - df_Tab2_tmp.loc[5, str(Year-1)+'年'+str(Month)+'月份发电量']) /\
                                                                df_Tab2_tmp.loc[5, str(Year-1)+'年'+str(Month)+'月份发电量']
df_Tab2_tmp.loc[5, str(Year)+'年'+str(Month-1)+'月份环比（%）'] = 100*(df_Tab2_tmp.loc[5, '本月发电量'] - df_Tab2_tmp.loc[5, str(Year)+'年'+str(Month-1)+'月份发电量']) /\
                                                                df_Tab2_tmp.loc[5, str(Year)+'年'+str(Month-1)+'月份发电量']
df_Tab2_tmp.rename(columns={'本月发电量':'本月发电量（亿千瓦时）'}, inplace=True)
# 选择性输出
print(str(Year)+'年'+str(Month)+'月冀北各地区风电发电量')
df_Tab2_tmp[['地区', '本月发电量（亿千瓦时）', '2017年7月份同比（%）', '2018年6月份环比（%）']]

2018年7月冀北各地区风电发电量


,地区,本月发电量（亿千瓦时）,2017年7月份同比（%）,2018年6月份环比（%）
1,张家口,9.967539,21.220961,-4.135788
2,承德,3.539847,16.225560,-28.179037
3,唐山,0.103110,18.258975,-26.591200
4,秦皇岛,0.068830,30.756079,-60.867588
5,冀北合计,13.679326,19.908679,-12.550928


### 2）表3-累计发电量

In [29]:
df_Combined.groupby(['地区']).sum()[['本月年累计发电量']]/10000.0

,本月年累计发电量
地区,
唐山,1.083510
张家口,102.884689
承德,56.914507
秦皇岛,1.100140


### 3）表6-利用小时

In [171]:
# 建立合并对照表头，“地区”字段用于显示，“地区平均”字段用于合并从《新能源处利用小时数》抽取的数据
df_TabIndex = pd.DataFrame({'地区': ['张家口', '承德', '唐山', '秦皇岛', '冀北合计'], '地区平均': ['张家口平均', '承德平均', '唐山平均', '秦皇岛平均', '全网风电']}, \
                           index=[1, 2, 3, 4, 5])

# 获取本年数据
df_Tab6_col1 = GetTable(['张家口平均', '承德平均', '唐山平均', '秦皇岛平均', '全网风电'], [str(Month)+'月利用小时'], FileNewEnergyDeparment)
df_Tab6_col2 = GetTable(['张家口平均', '承德平均', '唐山平均', '秦皇岛平均', '全网风电'], ['年累计利用小时'], FileNewEnergyDeparment)
df_Tab6_col2.rename(columns={'年累计利用小时':'累计利用小时'}, inplace=True)

# 获取去年同期数据，2017年数据列名与2018年不同，且2017年为“全网”， 2018年为“全网风电”！！！！请酌情手动修改
df_Tab6_col3 = GetTable(['张家口平均', '承德平均', '唐山平均', '秦皇岛平均', '全网'], ['年累计利用小时数'], FileNewEnergyDeparmentLastYear)
df_Tab6_col3.rename(columns={'年累计利用小时数':str(Year-1)+'年累计利用小时'}, inplace=True)
df_Tab6_col3.rename(index={'全网':'全网风电'}, inplace=True)

# 表格组合
df_Tab6_tmp = df_TabIndex.join(df_Tab6_col1, on='地区平均').join(df_Tab6_col2, on='地区平均').join(df_Tab6_col3, on='地区平均')

# 计算列
df_Tab6_tmp[str(Year-1)+'年同期偏差'] = df_Tab6_tmp['累计利用小时'] - df_Tab6_tmp[str(Year-1)+'年累计利用小时']
df_Tab6_tmp['累计利用小时同比'] = 100*(df_Tab6_tmp['累计利用小时'] - df_Tab6_tmp[str(Year-1)+'年累计利用小时']) / df_Tab6_tmp[str(Year-1)+'年累计利用小时']
df_Tab6_tmp['冀北平均水平偏差'] = df_Tab6_tmp['累计利用小时'] - df_Tab6_tmp.loc[5, '累计利用小时']

# 选择性输出
print('表7-冀北各地区光伏利用小时情况')
df_Tab6_tmp[['地区', str(Month)+'月利用小时', '累计利用小时', str(Year-1)+'年同期偏差', '累计利用小时同比', '冀北平均水平偏差']]

表7-冀北各地区光伏利用小时情况


,地区,7月利用小时,累计利用小时,2017年同期偏差,累计利用小时同比,冀北平均水平偏差
1,张家口,135.0,1406.133379,192.133379,15.826473,-36.812656
2,承德,95.0,1521.332194,227.332194,17.568176,78.386159
3,唐山,60.0,1189.063953,-57.936047,-4.646034,-253.882081
4,秦皇岛,71.0,1527.000000,202.000000,15.245283,84.053965
5,冀北合计,120.0,1442.946035,203.946035,16.460536,0.000000


### 4）表8-本月弃风

In [31]:
df_Tab8tmp = df_Combined.groupby(['地区']).sum()[['本月弃风电量', '本月发电量']]
df_Tab8 = df_Tab8tmp[['本月弃风电量']]
df_Tab8['本月弃风率'] = df_Tab8tmp['本月弃风电量'] / (df_Tab8tmp['本月弃风电量'] + df_Tab8tmp['本月发电量'])
df_Tab8

,本月弃风电量,本月弃风率
地区,,
唐山,0.00,0.000000
张家口,3050.28,0.029693
承德,239.50,0.006720
秦皇岛,0.00,0.000000


### 5）表9-累计弃风

In [32]:
df_Tab9tmp = df_Combined.groupby(['地区']).sum()[['本月年累计弃风电量', '本月年累计发电量']]
df_Tab9 = df_Tab9tmp[['本月年累计弃风电量']]
df_Tab9['本月年累计弃风率'] = df_Tab9tmp['本月年累计弃风电量'] / (df_Tab9tmp['本月年累计弃风电量'] + df_Tab9tmp['本月年累计发电量'])
df_Tab9

,本月年累计弃风电量,本月年累计弃风率
地区,,
唐山,2.00,0.000185
张家口,171841.18,0.143119
承德,24653.00,0.041517
秦皇岛,41.30,0.003740


In [169]:
df_Combined.groupby(['地区']).sum()

,上月年累计发电量,上月年累计利用小时数,上月年累计弃风电量,上月年累计弃风率,本月弃风电量,装机容量,本月发电量,本月年累计利用小时,本月年累计发电量,本月年累计弃风电量,本月年累计弃风率
地区,,,,,,,,,,,
唐山,9804.0,2749.108788,2.0,0.000314,0.00,17.200,1031.10,3034.478788,10835.10,2.00,0.000288
张家口,929171.5,61198.736286,168790.9,6.494395,3050.28,737.335,99675.39,67309.363462,1028846.89,171841.18,6.162906
承德,533746.6,33444.812275,24413.5,1.259998,239.50,371.635,35398.47,35914.165099,569145.07,24653.00,1.199671
秦皇岛,10313.1,2215.268939,41.3,0.007786,0.00,9.750,688.30,2356.288939,11001.40,41.30,0.007240
